In [8]:
# xMD testing
import pandas as pd
import os
from xMD.xMD import xMD
from xMD.MD_Settings import GROMACS_Settings

settings = GROMACS_Settings()


In [9]:




amber14sb_ff_path = os.path.join(os.getcwd())

# Set the GMXLIB environment variable
os.environ["GMXLIB"] = amber14sb_ff_path



In [10]:
settings = GROMACS_Settings()
settings.suffix = "APO_amber99"
# settings.search = "APO"

print(settings.config)

settings.topology = os.path.join(settings.topology,"BPTI")
print(settings.topology)
# make sure to turn on MPI for HPC 
settings.gmx_mpi_on = False


config
topology/BPTI


In [11]:
# list config dir 

configs = os.listdir(settings.config)

configs = [config if ("_" in config and ".mdp" in config) else None for config in configs]
configs = [config for config in configs if config is not None]

print(sorted(configs))


mdps = ['1_minim.mdp', '2_equil.mdp', '3_equil.mdp', '4_equil.mdp', '5_equil.mdp', '6_equil.mdp', '7_prod.mdp']

file_names = ["5PTI_APO_amber99_H_solv_ions.gro", "5PTI_APO_amber99_H.top", "5PTI_APO_amber99_H.itp"]

['1_minim.mdp', '2_equil.mdp', '3_equil.mdp', '4_equil.mdp', '5_equil.mdp', '6_equil.mdp', '7_prod.mdp', '7_prod_10ps_out.mdp', 'md_short.mdp']


In [12]:
# specify ARGS: -P, -R, -N

for rep in range(4,5+1):

    md = xMD(settings, 'BPTI_genvel4', "5PTI", rep)

    # md.check_args()
    md.create_directory_structure(overwrite=True)
    md.run_experiment(config_files=mdps,topology_files=file_names)
    save_path = md.save_experiment()

Replicate number:  4
Trial directory temporary:  temporary/MD/5PTI/BPTI_genvel4
Trial directory logs:  logs/MD/5PTI/BPTI_genvel4
Trial directory data:  data/MD/5PTI/BPTI_genvel4
Trial directory visualisation:  visualisation/MD/5PTI/BPTI_genvel4
Trial directory analysis:  analysis/MD/5PTI/BPTI_genvel4
Environment variables set:  GMXLIB /home/alexi/Documents/xMD
Trial directory temporary:  temporary/MD/5PTI/BPTI_genvel4
Trial directory logs:  logs/MD/5PTI/BPTI_genvel4
Trial directory data:  data/MD/5PTI/BPTI_genvel4
Trial directory visualisation:  visualisation/MD/5PTI/BPTI_genvel4
Trial directory analysis:  analysis/MD/5PTI/BPTI_genvel4
Trial directory temporary:  temporary/MD/5PTI/BPTI_genvel4
Trial directory logs:  logs/MD/5PTI/BPTI_genvel4
Trial directory data:  data/MD/5PTI/BPTI_genvel4
Trial directory visualisation:  visualisation/MD/5PTI/BPTI_genvel4
Trial directory analysis:  analysis/MD/5PTI/BPTI_genvel4
Creating directory:  R_1 Creating directory:  R_2 Creating directory:  R_3 

                      :-) GROMACS - gmx grompp, 2023.2 (-:

Executable:   /usr/local/gromacs/bin/gmx
Data prefix:  /usr/local/gromacs
Working dir:  /home/alexi/Documents/xMD
Command line:
  gmx grompp -f config/1_minim.mdp -c data/MD/5PTI/BPTI_genvel4/R_4/5PTI_APO_amber99_H_solv_ions.gro -p data/MD/5PTI/BPTI_genvel4/R_4/5PTI_APO_amber99_H.top -o data/MD/5PTI/BPTI_genvel4/R_4/APO_amber99_5PTI_0.tpr -r data/MD/5PTI/BPTI_genvel4/R_4/5PTI_APO_amber99_H_solv_ions.gro -maxwarn 1 -v

Ignoring obsolete mdp entry 'ns-type'

NOTE 1 [file config/1_minim.mdp]:
  You have set rlist larger than the interaction cut-off, but you also have
  verlet-buffer-tolerance > 0. Will set rlist using
  verlet-buffer-tolerance.


NOTE 2 [file config/1_minim.mdp]:
  Setting tcoupl from 'V-rescale' to 'no'. Temperature coupling does not
  apply to steep.


NOTE 3 [file config/1_minim.mdp]:
  Setting pcoupl from 'Parrinello-Rahman' to 'no'. Pressure coupling does
  not apply to steep.

Generating 1-4 interactions: f


Steepest Descents:
   Tolerance (Fmax)   =  1.00000e+02
   Number of steps    =        10000
Step=    0, Dmax= 1.0e-02 nm, Epot= -1.33807e+05 Fmax= 1.72031e+05, atom= 570
Step=    1, Dmax= 1.0e-02 nm, Epot= -1.70224e+05 Fmax= 6.60655e+04, atom= 10430
Step=    2, Dmax= 1.2e-02 nm, Epot= -1.99824e+05 Fmax= 2.39884e+04, atom= 20431
Step=    3, Dmax= 1.4e-02 nm, Epot= -2.24204e+05 Fmax= 8.65409e+03, atom= 13292
Step=    4, Dmax= 1.7e-02 nm, Epot= -2.45235e+05 Fmax= 4.39690e+03, atom= 20445
Step=    5, Dmax= 2.1e-02 nm, Epot= -2.59226e+05 Fmax= 1.66625e+04, atom= 569
Step=    6, Dmax= 2.5e-02 nm, Epot= -2.63172e+05 Fmax= 1.57266e+04, atom= 569
Step=    7, Dmax= 3.0e-02 nm, Epot= -2.64960e+05 Fmax= 2.52931e+04, atom= 569
Step=    8, Dmax= 3.6e-02 nm, Epot= -2.67414e+05 Fmax= 2.36781e+04, atom= 569
Step=   10, Dmax= 2.1e-02 nm, Epot= -2.70555e+05 Fmax= 5.72988e+03, atom= 569
Step=   11, Dmax= 2.6e-02 nm, Epot= -2.72732e+05 Fmax= 3.04115e+04, atom= 569
Step=   12, Dmax= 3.1e-02 nm, Epot= -2.7

Trajectory number:  1
checking input for internal consistency...

Setting the LD random seed to -18370561

processing topology...

Generated 2145 of the 2145 non-bonded parameter combinations

Generated 2145 of the 2145 1-4 parameter combinations

Excluding 3 bonded neighbours molecule type 'Protein_chain_A'

turning H bonds into constraints...

Excluding 2 bonded neighbours molecule type 'SOL'

turning H bonds into constraints...

Excluding 1 bonded neighbours molecule type 'NA'

turning H bonds into constraints...

Excluding 1 bonded neighbours molecule type 'CL'

turning H bonds into constraints...

processing coordinates...

double-checking input for internal consistency...

Setting gen_seed to -2491165

Velocities were taken from a Maxwell distribution at 300 K

Reading position restraint coords from data/MD/5PTI/BPTI_genvel4/R_4/APO_amber99_5PTI_0.gro

The center of mass of the position restraint coord's is  2.955  2.975  2.938

The center of mass of the position restraint coord'


NOTE 2 [file config/2_equil.mdp]:
  Removing center of mass motion in the presence of position restraints
  might cause artifacts. When you are using position restraints to
  equilibrate a macro-molecule, the artifacts are usually negligible.


There were 2 NOTEs

There was 1 WARNING

GROMACS reminds you: "We Don't Bother Anyone" (LIVE)

                      :-) GROMACS - gmx mdrun, 2023.2 (-:

Executable:   /usr/local/gromacs/bin/gmx
Data prefix:  /usr/local/gromacs
Working dir:  /home/alexi/Documents/xMD
Command line:
  gmx mdrun -v -deffnm data/MD/5PTI/BPTI_genvel4/R_4/APO_amber99_5PTI_1

Reading file data/MD/5PTI/BPTI_genvel4/R_4/APO_amber99_5PTI_1.tpr, VERSION 2023.2 (single precision)
Changing nstlist from 10 to 100, rlist from 0.8 to 0.859

1 GPU selected for this run.
Mapping of GPU IDs to the 2 GPU tasks in the 1 rank on this node:
  PP:0,PME:0
PP tasks will do (non-perturbed) short-ranged interactions on the GPU
PP task will update and constrain coordinates on the GPU
PME t

...

Last frame         20 time  200.000    ->  frame     19 time  190.000      
 ->  frame     20 time  200.000      
Last written: frame     20 time  200.000


GROMACS reminds you: "Does college pay? They do if you are a good open-field runner." (Will Rogers)

                     :-) GROMACS - gmx trjconv, 2023.2 (-:

Executable:   /usr/local/gromacs/bin/gmx
Data prefix:  /usr/local/gromacs
Working dir:  /home/alexi/Documents/xMD
Command line:
  gmx trjconv -f data/MD/5PTI/BPTI_genvel4/R_5/APO_amber99_5PTI_2-nojump.xtc -s data/MD/5PTI/BPTI_genvel4/R_5/APO_amber99_5PTI_2.tpr -dump 0 -o data/MD/5PTI/BPTI_genvel4/R_5/APO_amber99_5PTI_2-nojump.pdb

Will write pdb: Protein data bank file
Reading file data/MD/5PTI/BPTI_genvel4/R_5/APO_amber99_5PTI_2.tpr, VERSION 2023.2 (single precision)
Reading file data/MD/5PTI/BPTI_genvel4/R_5/APO_amber99_5PTI_2.tpr, VERSION 2023.2 (single precision)
Group     0 (         System) has 20445 elements
Group     1 (        Protein) has   892 elements
Group     

Note that major changes are planned in future for trjconv, to improve usability and utility.
Select group for output
Selected 1: 'Protein'
Running trjconv command 3:  ['gmx', 'trjconv', '-f', 'data/MD/5PTI/BPTI_genvel4/R_5/APO_amber99_5PTI_3-nojump.xtc', '-s', 'data/MD/5PTI/BPTI_genvel4/R_5/APO_amber99_5PTI_3.tpr', '-dump', '0', '-o', 'data/MD/5PTI/BPTI_genvel4/R_5/APO_amber99_5PTI_3-nojump.pdb']
Note that major changes are planned in future for trjconv, to improve usability and utility.
Select group for output
Selected 1: 'Protein'
Running trjconv command 1:  ['gmx', 'trjconv', '-f', 'data/MD/5PTI/BPTI_genvel4/R_5/APO_amber99_5PTI_4.xtc', '-s', 'data/MD/5PTI/BPTI_genvel4/R_5/APO_amber99_5PTI_4.tpr', '-pbc', 'mol', '-center', '-o', 'data/MD/5PTI/BPTI_genvel4/R_5/APO_amber99_5PTI_4-mol.xtc']
Note that major changes are planned in future for trjconv, to improve usability and utility.
Select group for centering
Selected 1: 'Protein'
Select group for output
Selected 0: 'System'
Running trj

Group     0 (         System) has 20445 elements
Group     1 (        Protein) has   892 elements
Group     2 (      Protein-H) has   454 elements
Group     3 (        C-alpha) has    58 elements
Group     4 (       Backbone) has   174 elements
Group     5 (      MainChain) has   233 elements
Group     6 (   MainChain+Cb) has   285 elements
Group     7 (    MainChain+H) has   289 elements
Group     8 (      SideChain) has   603 elements
Group     9 (    SideChain-H) has   221 elements
Group    10 (    Prot-Masses) has   892 elements
Group    11 (    non-Protein) has 19553 elements
Group    12 (          Water) has 19509 elements
Group    13 (            SOL) has 19509 elements
Group    14 (      non-Water) has   936 elements
Group    15 (            Ion) has    44 elements
Group    16 ( Water_and_ions) has 19553 elements
Reading frame       0 time    0.000   
Precision of data/MD/5PTI/BPTI_genvel4/R_5/APO_amber99_5PTI_5-nojump.xtc is 0.001 (nm)
Reading frame       1 time   10.000   
Du

Note that major changes are planned in future for trjconv, to improve usability and utility.
Select group for centering
Selected 1: 'Protein'
Select group for output
Selected 0: 'System'
Running trjconv command 2:  ['gmx', 'trjconv', '-f', 'data/MD/5PTI/BPTI_genvel4/R_5/APO_amber99_5PTI_6-mol.xtc', '-s', 'data/MD/5PTI/BPTI_genvel4/R_5/APO_amber99_5PTI_6.tpr', '-pbc', 'nojump', '-o', 'data/MD/5PTI/BPTI_genvel4/R_5/APO_amber99_5PTI_6-nojump.xtc']
Note that major changes are planned in future for trjconv, to improve usability and utility.
Select group for output
Selected 1: 'Protein'
Running trjconv command 3:  ['gmx', 'trjconv', '-f', 'data/MD/5PTI/BPTI_genvel4/R_5/APO_amber99_5PTI_6-nojump.xtc', '-s', 'data/MD/5PTI/BPTI_genvel4/R_5/APO_amber99_5PTI_6.tpr', '-dump', '0', '-o', 'data/MD/5PTI/BPTI_genvel4/R_5/APO_amber99_5PTI_6-nojump.pdb']
Note that major changes are planned in future for trjconv, to improve usability and utility.
Select group for output
Selected 1: 'Protein'
Note that ma

                     :-) GROMACS - gmx trjconv, 2023.2 (-:

Executable:   /usr/local/gromacs/bin/gmx
Data prefix:  /usr/local/gromacs
Working dir:  /home/alexi/Documents/xMD
Command line:
  gmx trjconv -f data/MD/5PTI/BPTI_genvel4/R_5/APO_amber99_5PTI_cat_5.xtc -s data/MD/5PTI/BPTI_genvel4/R_5/APO_amber99_5PTI_6-nojump.pdb -o visualisation/MD/5PTI/BPTI_genvel4/APO_amber99_5PTI_cat_5.pdb

Will write pdb: Protein data bank file
Group     0 (         System) has   892 elements
Group     1 (        Protein) has   892 elements
Group     2 (      Protein-H) has   454 elements
Group     3 (        C-alpha) has    58 elements
Group     4 (       Backbone) has   174 elements
Group     5 (      MainChain) has   233 elements
Group     6 (   MainChain+Cb) has   285 elements
Group     7 (    MainChain+H) has   289 elements
Group     8 (      SideChain) has   603 elements
Group     9 (    SideChain-H) has   221 elements
Reading frame       0 time    0.000   
Precision of data/MD/5PTI/BPTI_genvel4/R_

In [13]:
def parse_gromacs_energy_log(filepath):
    """
    Parse a GROMACS log file to extract energy information at each step into a DataFrame.
    """
   # read the file
    with open(filepath, 'r') as f:
        log = f.readlines()
    
    # find the start of the energy table

        for i, line in enumerate(log):
            #split by any whitespace
            split = line.split()
            # find:            Step           Time
            if len(split)==2 and split[0] == "Step" and split[1] == "Time":
                start = i
                break
            #find end

        for i, line in enumerate(log):
            #split by any whitespace
            split = line.split()
            if len(split)>1 and split[0] == "Statistics":
                end = i
                break

        # extract the table
        table = log[start+1:end]
    # extract the headers
    df = pd.DataFrame()

    return df


log_file_path = "data/MD/1K55/APO_MD60_genvel/R_1/APO_md_1K55_0.log"
# Parse the log file again with the updated function
energy_df_updated = parse_gromacs_energy_log(log_file_path)


FileNotFoundError: [Errno 2] No such file or directory: 'data/MD/1K55/APO_MD60_genvel/R_1/APO_md_1K55_0.log'